In [7]:
import pandas as pd 
import numpy as np

print(f"pandas version is: {pd.__version__}\nnumpy version is: {np.__version__}")

pandas version is: 2.3.1
numpy version is: 2.3.1


In [21]:
def load_and_filter_groups(filepath: str, min_members: int =200):
    """
    read and filter groups
    """

    df = pd.read_csv(filepath)

    # remove groups that name is empty and participants > min_members 
    df = df[df["peer_name"].notnull()]
    df = df[df["participants"] > min_members]


    df = df.drop_duplicates(subset="peerid")

    return df.head(5) 


In [ ]:
import re
def clean_text(text):
    """
    پاک‌سازی متن فارسی: حذف لینک‌ها، ایموجی، کاراکترهای اضافی
    """
    if pd.isnull(text):
        return ""
    # remove link
    text = re.sub(r"http\S+|www\S+|@\S+", "", text)
    

In [22]:
load_and_filter_groups(filepath="/mnt/d/Uni/thesis/data/idekav_subscription.csv")

,peerid,peer_name,about,type,participants
0,-2070440254,SafaBR,NaN,superchat,1501
136223,1000002630,جنگ آب,جمع صاحبنظران و دلسوختگان بحران آب ایران، برای...,superchat,1000
136224,1000004268,?BADANSAZAN?,⚜️ قوانین سوپر گروه بدنسازان ⚜️ \n? مطالب فقط...,superchat,377
136225,1000017302,شرکت جهان استیل@,09123716742 \n66752365,superchat,228
136226,1000018933,❀طب سنتی و علم پزشکی❀,?مخاطب گروه:آقايان و بانوان\n\n?موضوع:مطالب پز...,superchat,296
